In [68]:
import pandas as pd
import csv, sys, re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn import tree, metrics
from sklearn.utils import shuffle

df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE , dtype= str, header = None, names=["instance", "text", "id", "sentiment","is_sarcastic"])

In [69]:
df=shuffle(df)
df.head(3)
df.id.value_counts()

10003    358
10000    244
10005    194
10006    189
10008    163
10001    140
10002    130
10015    119
10013    104
10016     59
10010     56
10019     52
10017     47
10018     38
10014     29
10012     25
10004     17
10009     16
10011     13
10007      7
Name: id, dtype: int64

In [70]:
text_data = np.array([])

In [71]:
for text in df.text:
    text_data = np.append(text_data,text)

In [72]:
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
    punctuations = '''!"&'()*+,-./:;<=>?[\]^`{|}~'''
    no_punct = ""
    for char in sample:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct


def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = remove_punctuation(sample)
    return sample


def myTokenizer(sample):
    """Customized tokenizer"""
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2]
    return new_words

In [73]:
count = CountVectorizer(preprocessor=myPreprocessor, lowercase = False, tokenizer=myTokenizer ,max_features = 200)
bag_of_words = count.fit_transform(text_data)
print(count.get_feature_names())

['#AFPRaids', '#AusPol', '#AusVotes', '#Auspol', '#Ausvotes', '#Brexit', '#Election2016', '#LNP', '#LNPfail', '#Labor', '#Medicare', '#NBN', '#Parakeelia', '#auspol', '#ausvotes', '#ausvotes2016', '#insiders', '#npc', '#qanda', '@AustralianLabor', '@LiberalAus', '@RichardDiNatale', '@TurnbullMalcolm', '@billshortenmp', 'AFP', 'ALP', 'Abbott', 'Australia', 'Australian', 'Australians', 'Bill', 'Coalition', 'Dutton', 'Government', 'Govt', 'Greens', 'How', 'If', 'Its', 'Joyce', 'LNP', 'Labor', 'Labors', 'Liberal', 'Liberals', 'Libs', 'Malcolm', 'Medicare', 'Morrison', 'NBN', 'No', 'Not', 'PM', 'Party', 'Peter', 'Shorten', 'So', 'THE', 'The', 'This', 'Turnbull', 'Turnbulls', 'VOTE', 'Vote', 'We', 'What', 'Why', 'You', 'about', 'after', 'again', 'all', 'amp', 'an', 'and', 'any', 'are', 'as', 'at', 'back', 'be', 'because', 'been', 'big', 'boats', 'budget', 'business', 'but', 'by', 'campaign', 'can', 'cant', 'care', 'could', 'cut', 'cuts', 'did', 'do', 'dont', 'down', 'economic', 'economy', 'e

In [74]:
print(count.vocabulary_)

{'You': 67, 'will': 192, 'pay': 151, 'more': 135, 'to': 178, 'the': 170, 'under': 179, 'Liberals': 44, 'If': 37, 'you': 198, 'can': 90, 'it': 126, 'think': 175, 'of': 141, 'who': 190, '#ausvotes': 14, 'VOTE': 62, 'LNP': 40, '#auspol': 13, 'right': 159, 'Shorten': 55, 'that': 169, 'is': 125, '@LiberalAus': 20, 'our': 146, 'by': 88, 'like': 133, 'with': 193, 'this': 176, 'back': 79, 'election': 103, 'Greens': 35, 'could': 93, 'on': 142, 'boats': 84, 'for': 105, 'Labor': 41, 'The': 58, 'need': 137, 'take': 165, 'down': 99, 'and': 74, '#Auspol': 3, '#Labor': 9, 'put': 157, 'into': 124, 'cut': 94, 'Labors': 42, 'last': 132, 'has': 116, 'not': 139, 'Malcolm': 46, 'Turnbull': 60, 'are': 76, 'in': 123, 'his': 120, 'tax': 166, 'cuts': 95, 'plan': 153, '#AusVotes': 2, 'now': 140, '#insiders': 16, 'have': 117, '@TurnbullMalcolm': 22, 'be': 80, 'its': 127, 'get': 109, 'out': 147, 'Liberal': 43, 'no': 138, '#Parakeelia': 12, '#LNP': 7, '#AFPRaids': 0, '#qanda': 18, 'what': 188, 'say': 160, 'because

In [75]:
X = bag_of_words.toarray()

In [76]:
#target classes
Y = np.array([])
for text in df.id:
    Y = np.append(Y,text)

In [77]:
X_train = X[:1500]
X_test = X[1500:]
y_train = Y[:1500]
y_test = Y[1500:]

In [78]:
#clf = MultinomialNB()
#clf = BernoulliNB()
clf = tree.DecisionTreeClassifier(criterion='entropy',random_state=0) 
model = clf.fit(X_train, y_train)

In [79]:
predicted_y = model.predict(X_test)

In [80]:
print(accuracy_score(y_test, predicted_y))

0.258


In [82]:
print(y_test, predicted_y)
print(model.predict_proba(X_test))
print(accuracy_score(y_test, predicted_y))
print(precision_score(y_test, predicted_y, average='micro'))
print(recall_score(y_test, predicted_y, average='micro'))
print(f1_score(y_test, predicted_y, average='micro', labels = np.unique(predicted_y)))
print(f1_score(y_test, predicted_y, average='macro', labels = np.unique(predicted_y)))
print(classification_report(y_test, predicted_y,output_dict= False, labels = np.unique(predicted_y)))

['10018' '10003' '10002' '10001' '10003' '10017' '10006' '10005' '10015'
 '10013' '10005' '10002' '10017' '10003' '10005' '10005' '10003' '10000'
 '10003' '10008' '10015' '10005' '10005' '10003' '10014' '10008' '10008'
 '10006' '10019' '10005' '10001' '10008' '10003' '10006' '10005' '10003'
 '10012' '10002' '10003' '10012' '10003' '10004' '10003' '10001' '10003'
 '10016' '10003' '10001' '10002' '10001' '10000' '10013' '10001' '10016'
 '10003' '10006' '10015' '10008' '10019' '10000' '10003' '10002' '10003'
 '10000' '10008' '10003' '10006' '10015' '10008' '10003' '10003' '10005'
 '10018' '10001' '10005' '10001' '10002' '10005' '10008' '10006' '10006'
 '10005' '10019' '10001' '10017' '10000' '10008' '10008' '10000' '10016'
 '10005' '10000' '10003' '10005' '10003' '10000' '10008' '10008' '10017'
 '10006' '10013' '10005' '10008' '10000' '10010' '10005' '10008' '10003'
 '10006' '10005' '10005' '10005' '10003' '10003' '10000' '10000' '10001'
 '10018' '10005' '10008' '10005' '10000' '10003' '1

C:\Users\King\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\King\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
